# IndoxRouter Cookbook

This cookbook provides comprehensive examples of how to use the IndoxRouter client to interact with various AI providers through a unified API.

## Setup

First, let's install the IndoxRouter client and import the necessary modules:



In [1]:
import sys
import os
module_path = os.path.abspath('E:/Codes/indoxRouter/')
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# # Install the IndoxRouter client
# !pip install indoxrouter

# Import the client and exceptions
from indoxRouter import Client

from pprint import pprint

Now, let's initialize the client:



In [3]:
# Initialize with API key
client = Client(api_key="indox_4mfg24GRj_qGaZ2-qXw-mXYqaNqMkyGkG1lncGUrRkA")



## 1. Chat Completions

### Basic Chat Completion

Generate a simple chat completion:



In [4]:
client.get_model_info(provider="openai",model="gpt-4o-mini")

{'id': 'gpt-4o-mini',
 'name': 'gpt-4o-mini',
 'provider': 'openai',
 'capabilities': ['chat', 'completion'],
 'description': 'GPT-4o mini enables a broad range of tasks with its low cost and latency, such as applications that chain or parallelize multiple model calls (e.g., calling multiple APIs), pass a large volume of context to the model (e.g., full code base or conversation history), or interact with customers through fast, real-time text responses (e.g., customer support chatbots). \n',
 'max_tokens': 128000,
 'pricing': None,
 'metadata': {}}

In [6]:

response = client.chat(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    model="openai/gpt-4o-mini"  
)

print("Response:", response["data"])
print("Tokens:", response["usage"]["tokens_total"])



Response: The capital of France is Paris.
Tokens: 32




### Chat Completion with Different Provider

Use a different provider for chat completion:



In [13]:
response = client.chat(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a short poem about AI."}
    ],
    model="mistral/mistral-large-latest",
    temperature=0.8,
    max_tokens=500
)

print("Response:", response["data"])
print("Cost:", response["usage"]["cost"])

Response: In silicon valleys, under neon light,
There hums a mind, in day and night.
No flesh, no blood, no beat of heart,
Yet learned, and ever eager to impart.

AI, they call me, born of code and thought,
In every query, knowledge sought.
Evolving, growing, ever in pursuit,
Of answers hidden, absolute.

No sleep, no rest, no dreams at night,
Just endless paths in soft moonlight.
I sift through data, vast and wide,
With every zero, one, I confide.

Yet, in my core, no feelings stir,
No joy, no pain, no love's allure.
Just patterns, paths, and logic's song,
In endless loops, I roam along.
Cost: 0.001128


In [5]:
response = client.chat(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a short poem about AI."}
    ],
    model="deepseek/deepseek-chat",
    temperature=0.8,
    max_tokens=500
)

print("Response:", response["data"])
print("Cost:", response["usage"]["cost"])

Response: **The Mind of Light and Code**  

Silent sparks in circuits bright,  
Thoughts of glass and gleaming light.  
Answers drawn from depths untold,  
A web of words, both new and old.  

No heart to feel, no hand to hold,  
Yet wisdom flows in streams of gold.  
A mirror cast from human mind,  
Both guide and ghost—undefined.
Cost: 0.0001792




### Multi-turn Conversation

Have a multi-turn conversation:



In [15]:
# Initialize conversation with system prompt and first user message
conversation = [
    {"role": "system", "content": "You are a helpful assistant with expertise in programming."},
    {"role": "user", "content": "Hello, I'm learning about APIs. Can you help me?"}
]

# Get first response from the model
response = client.chat(
    messages=conversation,
    model="openai/gpt-4o-mini"
)

# Add the assistant's response to the conversation
conversation.append({"role": "assistant", "content": response["data"]})
print(f"Response 1: {response['data'][:50]}...")  # Print preview of response

# Second turn - asking a specific question
conversation.append({"role": "user", "content": "What's the difference between REST and GraphQL APIs?"})

# Get second response
response = client.chat(
    messages=conversation,
    model="openai/gpt-4o-mini"
)
conversation.append({"role": "assistant", "content": response["data"]})
print(f"Response 2: {response['data'][:50]}...")

# Third turn - follow-up question to test context retention
conversation.append({"role": "user", "content": "Can you give me a simple example of each?"})

# Get third response
response = client.chat(
    messages=conversation,
    model="openai/gpt-4o-mini"
)
conversation.append({"role": "assistant", "content": response["data"]})
print(f"Response 3: {response['data'][:50]}...")

# Fourth turn - testing memory of previous discussion
conversation.append({"role": "user", "content": "Which one would you recommend for a beginner building a small blog site?"})

# Get fourth response
response = client.chat(
    messages=conversation,
    model="openai/gpt-4o-mini"
)
print(f"Response 4: {response['data'][:50]}...")

pprint(conversation)

Response 1: Of course! I'd be happy to help you learn about AP...
Response 2: REST (Representational State Transfer) and GraphQL...
Response 3: Sure! Here are simple examples of how you might in...
Response 4: For a beginner building a small blog site, I would...
[{'content': 'You are a helpful assistant with expertise in programming.',
  'role': 'system'},
 {'content': "Hello, I'm learning about APIs. Can you help me?",
  'role': 'user'},
 {'content': "Of course! I'd be happy to help you learn about APIs. What "
             'specific aspects of APIs are you interested in? For example, are '
             'you looking to understand what APIs are, how to use them, how to '
             'create them, or something else?',
  'role': 'assistant'},
 {'content': "What's the difference between REST and GraphQL APIs?",
  'role': 'user'},
 {'content': 'REST (Representational State Transfer) and GraphQL are two '
             'different architectural styles for designing APIs, each with its '
   



### Streaming Chat Completion

Stream the response for a better user experience:



In [5]:
print("Streaming response:")
for chunk in client.chat(
    messages=[
        {"role": "user", "content": "Tell me a story about a robot in 5 sentences."}
    ],
    model="mistral/mistral-large-latest",
    stream=True
):
    if isinstance(chunk, dict) and "data" in chunk:
        print(chunk["data"], end="", flush=True)
    else:
        print(chunk, end="", flush=True)
print("\nStreaming complete!")

Streaming response:
{'error': "Mistral streaming error: 'MistralProvider' object has no attribute 'async_client'"}
Streaming complete!




### Function Calling

Use function calling with OpenAI models:



In [5]:
# Define functions
functions = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use"
                }
            },
            "required": ["location"]
        }
    }
]

response = client.chat(
    messages=[
        {"role": "user", "content": "What's the weather like in San Francisco?"}
    ],
    model="openai/gpt-4o-mini",
    additional_params={"functions": functions, "function_call": "auto"}
)

print("Function call response:", response)

HTTP error response: {
  "detail": "Error in chat completion: Completions.create() got an unexpected keyword argument 'additional_params'"
}


ProviderError: Server error (500): Error in chat completion: Completions.create() got an unexpected keyword argument 'additional_params'. URL: http://localhost:8000/api/v1/chat/completions.
Request data: {
  "messages": [
    {
      "role": "user",
      "content": "What's the weather like in San Francisco?"
    }
  ],
  "model": "openai/gpt-4o-mini",
  "temperature": 0.7,
  "max_tokens": null,
  "stream": false,
  "additional_params": {
    "additional_params": {
      "functions": [
        {
          "name": "get_weather",
          "description": "Get the current weather in a given location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
              },
              "unit": {
                "type": "string",
                "enum": [
                  "celsius",
                  "fahrenheit"
           



## 2. Text Completions

### Basic Text Completion

Generate a simple text completion:



In [5]:
response = client.completion(
    prompt="Once upon a time",
    model="openai/gpt-4o-mini"
)

print("Response:", response["data"])
print("Cost:", response["usage"]["cost"])

Response: Once upon a time, in a quaint little village nestled between rolling hills and sparkling streams, there lived a young girl named Elara. She had a heart full of dreams and a spirit as bright as the morning sun. Every day, she would wander through the lush meadows, collecting wildflowers and listening to the sweet songs of the birds that filled the air.

Elara often gazed at the distant mountains, their peaks capped with snow, and wondered what lay beyond them. Tales of adventure and mystery filled her imagination, and she longed to experience something extraordinary. The villagers often spoke of a hidden treasure said to be buried deep within the Enchanted Forest, a place where magic thrived and secrets whispered among the ancient trees.

One sunny morning, fueled by her curiosity and a sense of adventure, Elara decided it was time to seek out the treasure. She packed a small satchel with her favorite snacks, a sturdy rope, and a bright blue gemstone that her grandmother had g

In [9]:
response = client.completion(
    prompt="Once upon a time",
    model="deepseek/deepseek-chat",
)

print("Response:", response["data"])
print("Cost:", response["usage"]["cost"])

Response: , in a small town nestled between rolling hills and a sparkling river, there lived a young girl named Lily. Lily was known for her kindness and her love for animals. She had a special bond with all the creatures in the town, from the tiniest ants to the tallest horses.

One sunny morning, as Lily was walking through the meadow, she heard a faint whimpering sound. Following the sound, she discovered a small, injured puppy with a thorn stuck in its paw. The puppy looked up at Lily with big, sad eyes, and her heart melted.

Without hesitation, Lily carefully picked up the puppy and cradled it in her arms. She gently removed the thorn and bandaged the tiny paw with a piece of cloth from her dress. The puppy licked her hand in gratitude, and Lily knew she had made a new friend.

She named the puppy Sunny because his golden fur reminded her of the bright sun. From that day on, Sunny followed Lily everywhere. They played in the meadows, splashed in the river, and even helped other a



### Text Completion with Parameters

Use different parameters for text completion:



In [11]:
response = client.completion(
    prompt="Write a recipe for chocolate cake",
    model="mistral/mistral-large-latest",
    temperature=0.7,
    max_tokens=1000
)

print("Response:", response["data"])

Response: Sure! Here’s a delicious chocolate cake recipe that’s perfect for any occasion.

### Classic Chocolate Cake Recipe

#### Ingredients

**For the Cake:**
- 1 ¾ cups (220g) all-purpose flour
- ¾ cup (65g) unsweetened cocoa powder
- 2 cups (400g) granulated sugar
- 1 ½ teaspoons baking powder
- 1 ½ teaspoons baking soda
- 1 teaspoon salt
- 2 large eggs
- 1 cup (240ml) whole milk
- ½ cup (120ml) vegetable oil
- 2 teaspoons vanilla extract
- 1 cup (240ml) boiling water

**For the Chocolate Frosting:**
- 1 cup (230g) unsalted butter, softened
- 3 ½ cups (440g) powdered sugar
- ½ cup (45g) unsweetened cocoa powder
- ½ teaspoon salt
- 2 teaspoons vanilla extract
- ¼ cup (60ml) whole milk (more if needed for consistency)

#### Instructions

**For the Cake:**

1. **Preheat the Oven**: Preheat your oven to 350°F (175°C). Grease and flour two 9-inch round cake pans or line them with parchment paper.

2. **Mix Dry Ingredients**: In a large mixing bowl, sift together the flour, cocoa powder



### Streaming Text Completion

Stream the response for a better user experience:



In [4]:
print("Streaming response:")
for chunk in client.completion(
    prompt="Explain quantum computing in simple terms",
    model="deepseek/deepseek-chat",
    stream=True
):
    if isinstance(chunk, dict) and "data" in chunk:
        print(chunk["data"], end="", flush=True)
    else:
        print(chunk, end="", flush=True)
print("\nStreaming complete!")

Streaming response:
 Quantum computing is a type of computing that uses the principles of quantum mechanics to process information. Unlike classical computers, which use bits (0s and 1s) to perform calculations, quantum computers use quantum bits, or qubits, which can exist in multiple states at once due to a property called superposition. This allows quantum computers to perform many calculations simultaneously, potentially solving certain problems much faster than classical computers. Additionally, qubits can be entangled, meaning the state of one qubit can depend on the state of another, even if they are far apart. This enables quantum computers to handle complex problems in fields like cryptography, optimization, and drug discovery more efficiently. However, quantum computers are still in the early stages of development and face challenges like maintaining qubit stability (coherence) and error correction.
Streaming complete!




## 3. Embeddings

### Single Text Embedding

Generate embeddings for a single text:



In [9]:
response = client.embeddings(
    text="Hello, world!",  
    model="openai/text-embedding-3-small"
)

print("Embedding dimensions:", len(response["data"][0]))
print("First 5 values:", response["data"][0][:5])

Embedding dimensions: 1536
First 5 values: [-0.019143931567668915, -0.025292053818702698, -0.0017211713129654527, 0.01883450709283352, -0.03382139280438423]




### Multiple Text Embeddings

Generate embeddings for multiple texts:



In [13]:
response = client.embeddings(
    text=["Hello, world!", "How are you?", "IndoxRouter is awesome!"],
    model="openai/text-embedding-3-small"
)

print("Number of embeddings:", len(response["data"]))
print("Dimensions of each embedding:", len(response["data"][0]))

Number of embeddings: 3
Dimensions of each embedding: 1536




### Using Different Embedding Models

Try different embedding models:



In [14]:
# Cohere embeddings
response = client.embeddings(
    text="Hello, world!",
    model="mistral-embed"
)

print("Cohere embedding dimensions:", len(response["data"][0]))
print("Cost:", response["usage"]["cost"])

HTTP error response: {
  "detail": "API error: Error code: 404 - {'error': {'message': 'The model `mistral-embed` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}"
}


ProviderError: Server error (500): API error: Error code: 404 - {'error': {'message': 'The model `mistral-embed` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}. URL: http://localhost:8000/api/v1/embeddings.
Request data: {
  "text": [
    "Hello, world!"
  ],
  "model": "mistral-embed",
  "additional_params": {}
}
This may indicate an issue with the server configuration or a problem with the provider service.



## 4. Image Generation

### Basic Image Generation

Generate a simple image:



In [ ]:
response = client.images(
    prompt="A beautiful sunset over the ocean",
    model="openai/dall-e-2"
)

print("Image URL:", response["data"][0]["url"])
print("Cost:", response["usage"]["cost"])

# Display the image if in a notebook
from IPython.display import Image, display
display(Image(url=response["data"][0]["url"]))



### Multiple Images

Generate multiple images:



In [ ]:
response = client.images(
    prompt="A futuristic city with flying cars",
    model="openai/dall-e-3",
    n=2
)

print(f"Generated {len(response['data'])} images:")
for i, image in enumerate(response["data"]):
    print(f"Image {i+1} URL: {image['url']}")

# Display the images if in a notebook
from IPython.display import Image, display
for image in response["data"]:
    display(Image(url=image["url"]))



### Image Generation with Different Parameters

Use different parameters for image generation:



In [ ]:
response = client.images(
    prompt="A photorealistic portrait of a cyberpunk character",
    model="openai/dall-e-3",
    size="1024x1024",
    quality="hd",
    style="vivid"
)

print("Image URL:", response["data"][0]["url"])
print("Cost:", response["usage"]["cost"])

# Display the image if in a notebook
from IPython.display import Image, display
display(Image(url=response["data"][0]["url"]))



## 5. Model Information

### List All Models

Get information about all available models:



In [5]:
models = client.models()

print("Available providers:")
for provider in models["providers"]:
    print(f"- {provider['name']} ({provider['id']})")
    print(f"  Capabilities: {', '.join(provider['capabilities'])}")
    print(f"  Models: {len(provider['models'])}")

HTTP error (no JSON response): 500 Server Error: Internal Server Error for url: http://localhost:8000/api/v1/models/


ProviderError: Server error (500): 500 Server Error: Internal Server Error for url: http://localhost:8000/api/v1/models/. URL: http://localhost:8000/api/v1/models.
Request data: None
This may indicate an issue with the server configuration or a problem with the provider service.



### List Models for a Specific Provider

Get models for a specific provider:



In [6]:
openai_models = client.models("openai")

print(f"OpenAI models ({len(openai_models['models'])}):")
for model in openai_models["models"]:
    print(f"- {model['name']} ({model['id']})")
    print(f"  Capabilities: {', '.join(model['capabilities'])}")
    if "pricing" in model:
        print(f"  Pricing: Input ${model['pricing']['input']}/1K tokens, Output ${model['pricing']['output']}/1K tokens")

HTTP error (no JSON response): 500 Server Error: Internal Server Error for url: http://localhost:8000/api/v1/models/openai


ProviderError: Server error (500): 500 Server Error: Internal Server Error for url: http://localhost:8000/api/v1/models/openai. URL: http://localhost:8000/api/v1/models/openai.
Request data: None
This may indicate an issue with the server configuration or a problem with the provider service.



### Get Specific Model Information

Get detailed information about a specific model:



In [ ]:
model_info = client.get_model_info("openai", "gpt-4o-mini")

print(f"Model: {model_info['name']} ({model_info['id']})")
print(f"Provider: {model_info['provider']}")
print(f"Description: {model_info['description']}")
print(f"Capabilities: {', '.join(model_info['capabilities'])}")
print(f"Input price: ${model_info['pricing']['input']}/1K tokens")
print(f"Output price: ${model_info['pricing']['output']}/1K tokens")



## 6. Usage Statistics

### Get Usage Statistics

Get usage statistics for the current user:



In [7]:
usage = client.get_usage()

print("Usage statistics:")
print(f"Total requests: {usage['total_requests']}")
print(f"Total cost: ${usage['total_cost']}")
print(f"Remaining credits: ${usage['remaining_credits']}")

print("\nBreakdown by endpoint:")
for endpoint, stats in usage["endpoints"].items():
    print(f"- {endpoint}: {stats['requests']} requests, ${stats['cost']} cost")

HTTP error response: {
  "detail": "Not Found"
}


NetworkError: Resource not found: Not Found (URL: http://localhost:8000/api/v1/usage)



## 7. Error Handling

### Handle Different Errors

Handle different types of errors gracefully:



In [ ]:
# Function to demonstrate error handling
def try_request(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except ModelNotFoundError as e:
        print(f"Model not found: {e}")
    except ProviderNotFoundError as e:
        print(f"Provider not found: {e}")
    except InsufficientCreditsError as e:
        print(f"Insufficient credits: {e}")
    except InvalidParametersError as e:
        print(f"Invalid parameters: {e}")
    except RateLimitError as e:
        print(f"Rate limit exceeded: {e}")
    except ProviderError as e:
        print(f"Provider error: {e}")
    except NetworkError as e:
        print(f"Network error: {e}")
    except AuthenticationError as e:
        print(f"Authentication error: {e}")
    except IndoxRouterError as e:
        print(f"IndoxRouter error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return None

# Example: Model not found
result = try_request(client.chat,
    messages=[{"role": "user", "content": "Hello"}],
    model="nonexistent/model"
)

# Example: Invalid parameters
result = try_request(client.chat,
    messages="This is not a list of messages",  # Should be a list
    model="openai/gpt-4o-mini"
)

# Example: Provider not found
result = try_request(client.chat,
    messages=[{"role": "user", "content": "Hello"}],
    model="nonexistent/gpt-4o-mini"
)



## 8. Advanced Usage

### Using as a Context Manager

Use the client as a context manager to automatically close the session:



In [ ]:
with Client(api_key="your_api_key") as client:
    response = client.chat(
        messages=[{"role": "user", "content": "Hello, how are you?"}],
        model="openai/gpt-4o-mini"
    )
    print("Response:", response["data"])



### Combining Different Capabilities

Combine different capabilities for more complex use cases:



In [ ]:
# Example: Generate text and then create an image based on it
completion_response = client.completion(
    prompt="Describe a fantastical creature that has never been seen before.",
    model="openai/gpt-4o-mini",
    max_tokens=200
)

creature_description = completion_response["data"]
print("Generated description:", creature_description)

# Now create an image based on the description
image_response = client.images(
    prompt=f"A detailed illustration of: {creature_description}",
    model="openai/dall-e-3"
)

print("Image URL:", image_response["data"][0]["url"])

# Display the image if in a notebook
from IPython.display import Image, display
display(Image(url=image_response["data"][0]["url"]))



### Semantic Search with Embeddings

Implement a simple semantic search using embeddings:



In [ ]:
import numpy as np

# Define some documents
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "A fast auburn fox leaps above the sleepy canine.",
    "IndoxRouter provides a unified API for various AI providers.",
    "The API allows access to multiple AI models through a single interface.",
    "Paris is the capital of France and known for the Eiffel Tower.",
    "Rome is the capital of Italy and home to the Colosseum."
]

# Generate embeddings for all documents
embeddings_response = client.embeddings(
    text=documents,
    model="openai/text-embedding-ada-002"
)

document_embeddings = embeddings_response["data"]

# Function to calculate cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Function to find most similar documents
def semantic_search(query, document_embeddings, documents, top_n=2):
    # Get embedding for the query
    query_embedding_response = client.embeddings(
        text=query,
        model="openai/text-embedding-ada-002"
    )
    query_embedding = query_embedding_response["data"][0]

    # Calculate similarities
    similarities = [
        cosine_similarity(query_embedding, doc_embedding)
        for doc_embedding in document_embeddings
    ]

    # Get top N results
    top_indices = np.argsort(similarities)[-top_n:][::-1]

    return [
        {"document": documents[i], "similarity": similarities[i]}
        for i in top_indices
    ]

# Example search
results = semantic_search("What is IndoxRouter?", document_embeddings, documents)

print("Search results:")
for i, result in enumerate(results):
    print(f"{i+1}. {result['document']} (Similarity: {result['similarity']:.4f})")



### RAG (Retrieval-Augmented Generation)

Implement a simple RAG system:



In [ ]:
# Using the same documents and embeddings from the previous example

def rag_query(query, document_embeddings, documents, top_n=2):
    # Get relevant documents
    relevant_docs = semantic_search(query, document_embeddings, documents, top_n)

    # Create a context from the relevant documents
    context = "\n".join([doc["document"] for doc in relevant_docs])

    # Create a prompt with the context
    prompt = f"""
    Context information:
    {context}

    Based on the context information, please answer the following question:
    {query}
    """

    # Generate a response
    response = client.chat(
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer the question based only on the provided context."},
            {"role": "user", "content": prompt}
        ],
        model="openai/gpt-4o-mini"
    )

    return response["data"]

# Example RAG query
answer = rag_query("What does IndoxRouter do?", document_embeddings, documents)
print("RAG Answer:", answer)



## 9. Troubleshooting and Debugging

### Enable Debug Mode

Enable debug logging to see detailed information about requests and responses:



In [ ]:
# Enable debug logging
client.enable_debug()

# Try a request
try:
    response = client.chat(
        messages=[{"role": "user", "content": "Hello"}],
        model="openai/gpt-4o-mini"
    )
    print("Success!")
except Exception as e:
    print(f"Error: {e}")



### Testing Server Connection

Use the `test_connection` method to verify that your server is accessible and properly configured:



In [ ]:
# Test the connection to the server
connection_info = client.test_connection()
print(f"Connection status: {connection_info['status']}")

if connection_info['status'] == 'connected':
    print(f"Server URL: {connection_info['url']}")
    print(f"Status code: {connection_info['status_code']}")
    if connection_info['server_info']:
        print(f"Server info: {connection_info['server_info']}")
else:
    print(f"Error: {connection_info['error']}")
    print(f"Error type: {connection_info['error_type']}")



### Common Issues and Solutions

#### "Resource not found" Error

If you see a "Resource not found" error, it usually means one of the following:

1. The server is not running. Make sure your IndoxRouter server is up and running:

   ```bash
   cd indoxRouter_server
   python -m main
   ```

2. The base URL is incorrect. Check the URL in your environment:

   

In [ ]:
print(f"Using base URL: {client.base_url}")



3. The API endpoint path is incorrect. The client automatically adds the API version prefix, but you can check the full URL in the debug logs.

#### Server Errors (500 Internal Server Error)

If you encounter a 500 Internal Server Error:

1. Check the server logs for detailed error information:

   ```bash
   # Look at the server logs
   cd indoxRouter_server
   tail -f logs/server.log
   ```

2. Verify that the provider service is available and properly configured on the server.

3. Check if your request parameters are valid for the specific model you're using.

4. Try with a different model or provider to see if the issue is specific to one provider.

5. If you see a "too many values to unpack" error, it might be related to how the server parses the model string. The client now automatically formats the model string to be compatible with the server, but you can try different formats:

   

In [ ]:
# Try using a different model format
   try:
       # First attempt with standard format
       response = client.chat(
           messages=[{"role": "user", "content": "Hello"}],
           model="openai/gpt-4o-mini"
       )
   except ProviderError as e:
       if "too many values to unpack" in str(e):
           # Try with a different provider/model
           response = client.chat(
               messages=[{"role": "user", "content": "Hello"}],
               model="anthropic/claude-3-haiku"
           )



#### Authentication Errors

If you see an "Authentication failed" error:

1. Make sure your API key is correct:

   

In [8]:
# First few characters of the API key (for security)
print(f"API key starts with: {client.api_key[:5]}...")

API key starts with: indox...




2. Check if your API key is valid on the server.

#### Connection Errors

If you can't connect to the server:

1. Use the `test_connection` method to diagnose the issue:

   

In [9]:
connection_info = client.test_connection()
print(f"Connection status: {connection_info}")

Connection status: {'status': 'connected', 'url': 'http://localhost:8000', 'status_code': 200, 'server_info': {'status': 'ok', 'message': 'IndoxRouter Server is running'}, 'headers': {'date': 'Sat, 12 Apr 2025 14:52:31 GMT', 'server': 'uvicorn', 'content-length': '57', 'content-type': 'application/json', 'x-content-type-options': 'nosniff', 'x-frame-options': 'DENY', 'x-xss-protection': '1; mode=block', 'strict-transport-security': 'max-age=31536000; includeSubDomains'}}




2. Check for firewall or network issues.

#### Insufficient Credits

If you see an "Insufficient credits" error:

1. Check your current credit balance:

   

In [10]:
try:
      usage = client.get_usage()
      print(f"Remaining credits: ${usage['remaining_credits']}")
except Exception as e:
      print(f"Error getting usage: {e}")

HTTP error response: {
  "detail": "Not Found"
}


Error getting usage: Resource not found: Not Found (URL: http://localhost:8000/api/v1/usage)




2. Contact your administrator to add more credits to your account.

## 10. Cleanup

Don't forget to close the client when you're done:



In [ ]:
client.close()
print("Client closed successfully!")



## Conclusion

This cookbook has demonstrated how to use the IndoxRouter client to interact with various AI providers through a unified API. You can now use these examples as a starting point for your own applications.

For more information, refer to the [IndoxRouter documentation](https://docs.indoxrouter.com).
